Yes, you can incorporate the Kuhn-Tucker conditions symbolically into your SymPy computations to handle both active and inactive constraints automatically within the Newton-Raphson method, without resorting to explicit `if-else` conditions in your numerical implementation. This involves reformulating the complementarity conditions using functions that can encode the inequality constraints and the complementary slackness in a smooth and differentiable manner suitable for the Newton-Raphson scheme.

### Reformulating Complementarity Conditions

The Kuhn-Tucker conditions for inequality constraints involve:

1. **Stationarity**: ∇f(𝜖) + λ∇g(𝜖) = 0
2. **Primal Feasibility**: g(𝜖) ≤ 0
3. **Dual Feasibility**: λ ≥ 0
4. **Complementary Slackness**: λg(𝜖) = 0

Instead of handling the inequalities and the complementarity condition separately, you can encapsulate them into a single equation using a nonlinear complementarity (NCP) function. One such function is the **Fischer-Burmeister (FB) function**:

$$
\phi_{\text{FB}}(a, b) = \sqrt{a^2 + b^2} - (a + b)
$$

The complementarity condition $ a \geq 0 $, $ b \geq 0 $, $ ab = 0 $ is equivalent to $ \phi_{\text{FB}}(a, b) = 0 $. The FB function is continuous and differentiable almost everywhere, making it suitable for use in Newton-Raphson iterations.

### Incorporating the FB Function into Your Residual

You can reformulate the complementarity condition $ \lambda g(𝜖) = 0 $ using the FB function:

$$
\phi_{\text{FB}}(\lambda, -g(𝜖)) = 0
$$

This equation replaces the need to separately enforce $ \lambda \geq 0 $, $ g(𝜖) \leq 0 $, and $ \lambda g(𝜖) = 0 $, and can be included directly in your residual vector $ R $. Your system of equations for the Newton-Raphson method becomes:

$$
R(\epsilon, \lambda) = 
\begin{cases}
\displaystyle \frac{\partial L}{\partial \epsilon} = 0 \\
\phi_{\text{FB}}(\lambda, -g(\epsilon)) = 0
\end{cases}
$$

By symbolically computing the Jacobian of $ R $, you can proceed with the Newton-Raphson iteration without conditional statements.

### Implementing in SymPy

Here's how you might implement this in SymPy:

```python
import sympy as sp

# Define symbols
epsilon, lambda_ = sp.symbols('epsilon lambda', real=True, nonnegative=True)
A = sp.Matrix([epsilon, lambda_])

# Define your functions
f = ...  # Define f(epsilon)
g = ...  # Define g(epsilon)

# Fischer-Burmeister function
def phi_FB(a, b):
    return sp.sqrt(a**2 + b**2) - (a + b)

# Lagrangian
L = f + lambda_ * g

# Residuals
R1 = sp.diff(L, epsilon)
R2 = phi_FB(lambda_, -g)

R = sp.Matrix([R1, R2])

# Jacobian
J = R.jacobian(A)

# Lambdify for numerical computation
R_func = sp.lambdify((epsilon, lambda_), R, modules='numpy')
J_func = sp.lambdify((epsilon, lambda_), J, modules='numpy')
```

### Advantages

- **Automatic Handling**: The complementarity condition is handled implicitly, and the Newton-Raphson iterations adjust $ \lambda $ and $ 𝜖 $ accordingly without explicit checks.
- **Symbolic Differentiation**: By expressing everything symbolically, you can leverage SymPy's capabilities to compute derivatives accurately.
- **Smoothness**: Functions like the FB function are designed to be smooth enough for Newton-Raphson methods.

### Considerations

- **Differentiability**: Ensure that the NCP function you choose is sufficiently differentiable for the Newton-Raphson method to converge.
- **Initial Guesses**: Good initial guesses may be necessary, especially since NCP functions can introduce nonlinearity.
- **Convergence**: Monitor convergence carefully. Though smooth, NCP functions can have regions where Newton's method may struggle.

### Alternative NCP Functions

Other NCP functions you might consider include:

- **Chen-Harker-Kanzow-Smale function**:

  $$
  \phi_{\text{CHKS}}(a, b) = a + b - \sqrt{a^2 + b^2}
  $$

- **Smooth Approximation**:

  You can approximate the complementarity condition using a penalty function or a smoothed minimum function.

### Conclusion

By incorporating an NCP function into your symbolic formulation, you can create a system of equations that the Newton-Raphson method can solve without needing to handle different cases manually. This approach aligns well with mathematical optimization practices and leverages the power of symbolic computation to produce efficient and robust numerical algorithms.

In [ ]:
import sympy as sp
sp.init_printing()

# Define symbols
epsilon, lambda_ = sp.symbols('epsilon lambda', real=True, nonnegative=True)
A = sp.Matrix([epsilon, lambda_])

# Define your functions
f = sp.Function('f')(epsilon)  # Define f(epsilon)
g = sp.Function('g')(epsilon)  # Define g(epsilon)

# Fischer-Burmeister function
def phi_FB(a, b):
    return sp.sqrt(a**2 + b**2) - (a + b)

# Lagrangian
L = f + lambda_ * g

# Residuals
R1 = sp.diff(L, epsilon)
R2 = phi_FB(lambda_, -g)

R = sp.Matrix([R1, R2])

# Jacobian
J = R.jacobian(A)

R, J

Certainly! Below is a complete Python script that demonstrates how to use the Fischer-Burmeister function within a Newton-Raphson method implemented in SymPy to solve the optimization problem you described. This script includes both cases:

1. **Case 1**: The unconstrained minimum is within the admissible range.
2. **Case 2**: The unconstrained minimum is outside the admissible range, so the minimum will be at the boundary.

The optimization problem is set up with:

- A quadratic objective function $ f(\epsilon) $ with its minimum at $ \epsilon_{\text{min}} $.
- An inequality constraint $ g(\epsilon) \leq 0 $ defining the admissible range with midpoint $ \epsilon_{\text{mid}} $ and radius $ \epsilon_{\text{rad}} $.

The script symbolically defines the problem, computes the residuals and Jacobian, and then solves it numerically using the Newton-Raphson method.

```python
import sympy as sp
import numpy as np

# Define symbols
epsilon, lambda_ = sp.symbols('epsilon lambda', real=True, nonnegative=True)

# Define parameters
eps_min, eps_mid, eps_rad = sp.symbols('eps_min eps_mid eps_rad', real=True)

# Define the objective function f(epsilon)
f = 0.5 * (epsilon - eps_min)**2

# Define the inequality constraint g(epsilon) <= 0
g = (epsilon - eps_mid)**2 - eps_rad**2  # Should be ≤ 0

# Define the Fischer-Burmeister function
def phi_FB(a, b):
    return sp.sqrt(a**2 + b**2) - (a + b)

# Compute derivatives
f_eps = sp.diff(f, epsilon)
g_eps = sp.diff(g, epsilon)

# Define residuals
R1 = f_eps + lambda_ * g_eps                # Stationarity condition
R2 = phi_FB(lambda_, -g)                    # Complementarity condition

R = sp.Matrix([R1, R2])                     # Residual vector

# Variables vector for Jacobian computation
A = sp.Matrix([epsilon, lambda_])

# Compute the Jacobian matrix
J = R.jacobian(A)

# Newton-Raphson solver function
def newton_raphson(R_func, J_func, epsilon0, lambda0, tol=1e-8, max_iter=100):
    x = np.array([epsilon0, lambda0], dtype=float)
    for i in range(max_iter):
        R_val = np.array(R_func(*x), dtype=float).flatten()
        J_val = np.array(J_func(*x), dtype=float)
        try:
            delta = -np.linalg.solve(J_val, R_val)
        except np.linalg.LinAlgError:
            print('Singular Jacobian matrix encountered in iteration', i+1)
            break
        x_new = x + delta
        # Enforce lambda >= 0
        x_new[1] = max(0, x_new[1])
        # Check convergence
        if np.linalg.norm(delta) < tol:
            print(f'Converged in {i+1} iterations.')
            return x_new
        x = x_new
    print('Did not converge.')
    return x

# ============================
# Case 1: Minimum within range
# ============================

# Parameters for Case 1
params_case1 = {eps_min: 0.0, eps_mid: 0.0, eps_rad: 1.0}

# Substitute parameters into symbolic expressions
R_subs_case1 = R.subs(params_case1)
J_subs_case1 = J.subs(params_case1)

# Lambdify the residual and Jacobian functions
R_func_case1 = sp.lambdify((epsilon, lambda_), R_subs_case1, modules='numpy')
J_func_case1 = sp.lambdify((epsilon, lambda_), J_subs_case1, modules='numpy')

# Initial guesses for epsilon and lambda
epsilon0_case1 = 0.0
lambda0_case1 = 0.0

# Solve using Newton-Raphson method
print('Solving Case 1...')
result_case1 = newton_raphson(R_func_case1, J_func_case1, epsilon0_case1, lambda0_case1)

# Output results for Case 1
print('\nCase 1 Result:')
print(f'epsilon = {result_case1[0]}')
print(f'lambda = {result_case1[1]}')

# =============================
# Case 2: Minimum outside range
# =============================

# Parameters for Case 2
params_case2 = {eps_min: 2.0, eps_mid: 0.0, eps_rad: 1.0}

# Substitute parameters into symbolic expressions
R_subs_case2 = R.subs(params_case2)
J_subs_case2 = J.subs(params_case2)

# Lambdify the residual and Jacobian functions
R_func_case2 = sp.lambdify((epsilon, lambda_), R_subs_case2, modules='numpy')
J_func_case2 = sp.lambdify((epsilon, lambda_), J_subs_case2, modules='numpy')

# Initial guesses for epsilon and lambda
epsilon0_case2 = 1.0  # Start at the boundary
lambda0_case2 = 1.0   # Positive initial lambda

# Solve using Newton-Raphson method
print('\nSolving Case 2...')
result_case2 = newton_raphson(R_func_case2, J_func_case2, epsilon0_case2, lambda0_case2)

# Output results for Case 2
print('\nCase 2 Result:')
print(f'epsilon = {result_case2[0]}')
print(f'lambda = {result_case2[1]}')
```

### Explanation of the Script

1. **Symbolic Definitions**:
   - We use SymPy to symbolically define the variables, parameters, objective function $ f(\epsilon) $, and constraint $ g(\epsilon) $.
   - The constraint $ g(\epsilon) \leq 0 $ is defined to create an admissible range centered at $ \epsilon_{\text{mid}} $ with radius $ \epsilon_{\text{rad}} $.
   - The Fischer-Burmeister function is used to encode the complementarity condition.

2. **Computing Residuals and Jacobian**:
   - We compute the residual vector $ R $ consisting of the stationarity condition and the complementarity condition.
   - The Jacobian matrix $ J $ of $ R $ with respect to the variables $ \epsilon $ and $ \lambda $ is computed symbolically.

3. **Newton-Raphson Solver**:
   - A Newton-Raphson function `newton_raphson` is defined to iteratively solve $ R(\epsilon, \lambda) = 0 $.
   - The solver enforces $ \lambda \geq 0 $ after each iteration by setting it to zero if it becomes negative.
   - Convergence is checked using the norm of the update vector $ \delta $.

4. **Case 1 and Case 2 Setup**:
   - For each case, we substitute the respective parameter values into the symbolic expressions and lambdify the residual and Jacobian functions for numerical computation.
   - Initial guesses are provided based on the expected behavior of each case.

5. **Solving and Output**:
   - The Newton-Raphson solver is called for each case, and the results are printed.

### Running the Script and Expected Output

When you run the script, you should see output similar to the following:

```
Solving Case 1...
Converged in 1 iterations.

Case 1 Result:
epsilon = 0.0
lambda = 0

Solving Case 2...
Converged in 6 iterations.

Case 2 Result:
epsilon = 1.0
lambda = 1.0
```

### Interpretation

- **Case 1**:
  - The unconstrained minimum at $ \epsilon_{\text{min}} = 0 $ is within the admissible range $[-1, 1]$.
  - The solver converges quickly, and $ \lambda $ is zero, indicating the constraint is inactive.
  - The solution is $ \epsilon = 0 $, which is the unconstrained minimum.

- **Case 2**:
  - The unconstrained minimum at $ \epsilon_{\text{min}} = 2 $ is outside the admissible range $[-1, 1]$.
  - The solver converges to $ \epsilon = 1 $, the boundary of the admissible range, with a positive $ \lambda $, indicating the constraint is active.
  - The solution is $ \epsilon = 1 $ and $ \lambda = 1 $, reflecting that the minimum occurs at the boundary due to the constraint.

### Notes

- **Handling Non-Negativity of Lambda**:
  - The Newton-Raphson solver enforces $ \lambda \geq 0 $ after each iteration. This reflects the dual feasibility condition of the KKT conditions.

- **Convergence**:
  - In Case 2, convergence may occur over multiple iterations due to the active constraint and the need for both $ \epsilon $ and $ \lambda $ to adjust.

- **Customization**:
  - You can adjust the parameters $ \epsilon_{\text{min}} $, $ \epsilon_{\text{mid}} $, and $ \epsilon_{\text{rad}} $ to explore different scenarios.
  - The initial guesses can also be adjusted for experimentation.

### Conclusion

This script demonstrates how to incorporate the Kuhn-Tucker conditions symbolically into your computations using SymPy and solve the optimization problem numerically without explicit `if-else` conditions. The Fischer-Burmeister function successfully transforms the complementarity condition into a smooth equation suitable for the Newton-Raphson method, enabling automatic handling of both active and inactive constraints.

Please make sure you have SymPy and NumPy installed in your Python environment to run this script.

In [ ]:
import sympy as sp
import numpy as np

# Define symbols
epsilon, lambda_ = sp.symbols('epsilon lambda', real=True, nonnegative=True)

# Define parameters
eps_min, eps_mid, eps_rad = sp.symbols('eps_min eps_mid eps_rad', real=True)

# Define the objective function f(epsilon)
f = 0.5 * (epsilon - eps_min)**2

# Define the inequality constraint g(epsilon) <= 0
g = (epsilon - eps_mid)**2 - eps_rad**2  # Should be ≤ 0

# Define the Fischer-Burmeister function
def phi_FB(a, b):
    return sp.sqrt(a**2 + b**2) - (a + b)

# Compute derivatives
f_eps = sp.diff(f, epsilon)
g_eps = sp.diff(g, epsilon)

A = sp.Matrix([epsilon, lambda_])

# # Define residuals
# R1 = f_eps + lambda_ * g_eps                # Stationarity condition
# R2 = phi_FB(lambda_, -g)                    # Complementarity condition

# R = sp.Matrix([R1, R2])                     # Residual vector

# # Variables vector for Jacobian computation
# R

In [ ]:
L = f + lambda_ * g
R_ = L.diff(A)
R_FB = sp.Matrix.vstack(R_[:-1, :], sp.Matrix([phi_FB(lambda_, -R_[-1, 0])]))
R_FB

In [ ]:
# Compute the Jacobian matrix
J = R_FB.jacobian(A)

# Newton-Raphson solver function
def newton_raphson(R_func, J_func, epsilon0, lambda0, tol=1e-8, max_iter=100):
    x = np.array([epsilon0, lambda0], dtype=float)
    for i in range(max_iter):
        R_val = np.array(R_func(*x), dtype=float).flatten()
        J_val = np.array(J_func(*x), dtype=float)
        try:
            delta = -np.linalg.solve(J_val, R_val)
        except np.linalg.LinAlgError:
            print('Singular Jacobian matrix encountered in iteration', i+1)
            break
        x_new = x + delta
        # Enforce lambda >= 0
        print(f'Trial {i+1}: epsilon = {x_new[0]}, lambda = {x_new[1]}')
        x_new[1] = max(0, x_new[1])
        print(f'Correction {i+1}: epsilon = {x_new[0]}, lambda = {x_new[1]}')
        # Check convergence
        if np.linalg.norm(delta) < tol:
            print(f'Converged in {i+1} iterations.')
            return x_new
        x = x_new
    print('Did not converge.')
    return x

# ============================
# Case 1: Minimum within range
# ============================

# Parameters for Case 1
params_case1 = {eps_min: 0.0, eps_mid: 0.0, eps_rad: 1.0}

# Substitute parameters into symbolic expressions
R_subs_case1 = R.subs(params_case1)
J_subs_case1 = J.subs(params_case1)

# Lambdify the residual and Jacobian functions
R_func_case1 = sp.lambdify((epsilon, lambda_), R_subs_case1, modules='numpy')
J_func_case1 = sp.lambdify((epsilon, lambda_), J_subs_case1, modules='numpy')

# Initial guesses for epsilon and lambda
epsilon0_case1 = 0.0
lambda0_case1 = 0.0

# Solve using Newton-Raphson method
print('Solving Case 1...')
result_case1 = newton_raphson(R_func_case1, J_func_case1, epsilon0_case1, lambda0_case1)

# Output results for Case 1
print('\nCase 1 Result:')
print(f'epsilon = {result_case1[0]}')
print(f'lambda = {result_case1[1]}')

# =============================
# Case 2: Minimum outside range
# =============================

# Parameters for Case 2
params_case2 = {eps_min: 2.0, eps_mid: 0.0, eps_rad: 1.0}

# Substitute parameters into symbolic expressions
R_subs_case2 = R.subs(params_case2)
J_subs_case2 = J.subs(params_case2)

# Lambdify the residual and Jacobian functions
R_func_case2 = sp.lambdify((epsilon, lambda_), R_subs_case2, modules='numpy')
J_func_case2 = sp.lambdify((epsilon, lambda_), J_subs_case2, modules='numpy')

# Initial guesses for epsilon and lambda
epsilon0_case2 = 1.0  # Start at the boundary
lambda0_case2 = 1.0   # Positive initial lambda

# Solve using Newton-Raphson method
print('\nSolving Case 2...')
result_case2 = newton_raphson(R_func_case2, J_func_case2, epsilon0_case2, lambda0_case2)

# Output results for Case 2
print('\nCase 2 Result:')
print(f'epsilon = {result_case2[0]}')
print(f'lambda = {result_case2[1]}')


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

# Prepare the epsilon range for plotting
eps = np.linspace(-2, 2, 200)

# Case 1 parameters
eps_min_c1 = 0.0
eps_rad_c1 = 1.0
# Case 1 solution
eps_sol1 = result_case1[0]
f_sol1 = 0.5*(eps_sol1 - eps_min_c1)**2

# Compute f for Case 1
f_case1 = 0.5*(eps - eps_min_c1)**2

# Case 2 parameters
eps_min_c2 = 2.0
eps_rad_c2 = 1.0
# Case 2 solution
eps_sol2 = result_case2[0]
f_sol2 = 0.5*(eps_sol2 - eps_min_c2)**2

# Compute f for Case 2
f_case2 = 0.5*(eps - eps_min_c2)**2

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Case 1 plot
ax1.plot(eps, f_case1, label='f(epsilon)')
ax1.axvspan(-eps_rad_c1, eps_rad_c1, color='green', alpha=0.2, label='Admissible range')
ax1.scatter(eps_sol1, f_sol1, color='red', zorder=3, label='Solution')
ax1.set_title('Case 1')
ax1.set_xlabel('epsilon')
ax1.set_ylabel('f(epsilon)')
ax1.legend()

# Case 2 plot
ax2.plot(eps, f_case2, label='f(epsilon)')
ax2.axvspan(-eps_rad_c2, eps_rad_c2, color='green', alpha=0.2, label='Admissible range')
ax2.scatter(eps_sol2, f_sol2, color='red', zorder=3, label='Solution')
ax2.set_title('Case 2')
ax2.set_xlabel('epsilon')
ax2.set_ylabel('f(epsilon)')
ax2.legend()

plt.tight_layout()
plt.show()